In [52]:
from keras_contrib.layers.normalization import InstanceNormalization, BatchRenormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate, merge, Conv2DTranspose, Add
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import ReLU, LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.engine.topology import Layer
from keras.engine import InputSpec
import tensorflow as tf

In [2]:
class ReflectionPadding2D(Layer):
    """
    from https://stackoverflow.com/questions/50677544/reflection-padding-conv2d
    """
    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        self.input_spec = [InputSpec(ndim=4)]
        super(ReflectionPadding2D, self).__init__(**kwargs)

    def get_output_shape_for(self, s):
        """ If you are using "channels_last" configuration"""
        return (s[0], s[1] + 2 * self.padding[0], s[2] + 2 * self.padding[1], s[3])

    def call(self, x, mask=None):
        w_pad,h_pad = self.padding
        return tf.pad(x, [[0,0], [h_pad,h_pad], [w_pad,w_pad], [0,0] ], 'REFLECT')

In [3]:
width          = 1024
height         = 1024
input_channel  = 2
output_channel = 1
image_shape    = (width, height, input_channel)

In [12]:
def ck(layer_input, filters, f_size=7, i_norm=True) :
    d = ReflectionPadding2D(padding=(3, 3))(layer_input)
    d = Conv2D(filters, kernel_size=f_size, padding="valid")(d)
    if i_norm == True :
        d = InstanceNormalization()(d)
    d = ReLU()(d)
    return d
def dk(layer_input, filters, f_size=3, i_norm=True) :
    d = Conv2D(filters, kernel_size=f_size, strides=2, padding="same")(layer_input)
    if i_norm == True :
        d = InstanceNormalization()(d)
    d = ReLU()(d)
    return d
def Rk(layer_input, filters, f_size=3, i_norm = True) :
    d = ReflectionPadding2D(padding=(1, 1))(layer_input)
    d = Conv2D(filters, kernel_size=f_size, padding="valid")(d)

    if i_norm == True :
        d = InstanceNormalization()(d)
    d = ReLU()(d)
    d = ReflectionPadding2D(padding=(1, 1))(d)
    d = Conv2D(filters, kernel_size=f_size, padding="valid")(d)
    if i_norm == True :
        d = InstanceNormalization()(d)
    d = merge.add([layer_input, d])
    d = ReLU()(d)
    return d
def uk(layer_input, filters, f_size=3, i_norm=True) :
    d = Conv2DTranspose(filters, kernel_size=f_size, stride=2, padding="same", output_padding="same")(layer_input)
    if i_norm == True:
        d = InstanceNormalization()(d)
    d = ReLU()(d)
    return d

In [20]:

gn_shape = (image_shape[0] / 2, image_shape[1] / 2, image_shape[2])

g0 = Input(gn_shape)
g1 = ck(g0, 64)
g2 = dk(g1, 128)
g3 = dk(g2, 256)
g4 = dk(g3, 512)
g5 = dk(g4, 1024)

TensorShape([Dimension(None), Dimension(32), Dimension(32), Dimension(1024)])

In [61]:
d = ReflectionPadding2D(padding=(1, 1))(g5)
d = Conv2D(1024, kernel_size=3, padding="valid")(d)
d = InstanceNormalization()(d)
d = ReLU()(d)
d = ReflectionPadding2D(padding=(1, 1))(d)
d = Conv2D(1024, kernel_size=3, padding="valid")(d)
d = InstanceNormalization()(d)
print(d.shape, g5.shape)
d = Add()([g5, d])
d = ReLU()(d)

(?, 32, 32, 1024) (?, 32, 32, 1024)


ValueError: Operands could not be broadcast together with shapes (32.0, 32.0, 1024) (28.0, 28.0, 1024)


g7 = Rk(g6, 1024)
g8 = Rk(g7, 1024)
g9 = Rk(g8, 1024)
g10 = Rk(g9, 1024)
g11 = Rk(g10, 1024)
g12 = Rk(g11, 1024)
g13 = Rk(g12, 1024)
g14 = Rk(g13, 1024)
g15 = uk(g14, 512)
g16 = uk(g15, 256)
g17 = uk(g16, 128)
g18 = uk(g17, 64)